<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracció dels vectors de característiques

Aquest fitxer està pensat per ser utilitzat després dels entrenaments fets
en el fitxer de EvaluacioArquitectures. Utilitzarem els pesos del model elegit per a guardar els vectors de característiques de les imatges pertanyents al conjunt d'entrenament. Aquests vectors es guardaran dins una carpeta anomenada 'Features' que es situarà dins la mateixa carpeta dels pesos del model.

D'aquesta manera tindrem una carpeta per a cada model. De cada model hi tindrem una carpeta numerada per a cada execució i, dins cada una d'aquestes, hi guardarem els vectors de característiques a la carpeta 'Features'.

In [ ]:
import torch
import cv2
import csv
import random
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl
import json

from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io
from google.colab import drive

DOWNLOAD = False


#Name of the trained models
MODELS = {
    0: 'AlexnetBinary',
    1: 'Resnet152Binary',
    2: 'Inceptionv3Binary',
    3: 'EfficientNetB1Binary',
    4: 'AlexnetMulticlass',
    5: 'Resnet152Multiclass',
    6: 'Inceptionv3Multiclass',
    7: 'EfficientnetB1Multiclass',
}


MODEL = MODELS[6] #triam un model
RUN = 9 #triam el nombre de l'execució a la que s'han guardat els pesos (el nombre de la carpeta)


WEIGHTS_PATH = "/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/" #camí dels pesos
TRAINING_INFO = "/content/drive/MyDrive/Runs/Ham10000-"+MODEL+"/"+str(RUN)+"/training_info.json" #camí de les dades guardades de l'entrenament
#/content/drive/MyDrive/Runs/Ham10000Resnet152Multiclass/5/pesos_resnet152multiclass_5.pt
pt_file = glob(WEIGHTS_PATH+"*.pt") #fitxer de pesos

drive.mount('/content/drive')


if DOWNLOAD:
  !rm -rf /content/sample_data/*

  !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

  !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

  !kaggle datasets download -d "tschandl/ham10000-lesion-segmentations"

  !unzip -o ham10000-lesion-segmentations.zip -d /content/sample_data/

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


A continuació cercam quins paràmetres hem utilitzat en el conjunt d'entrenament per a replicar el mateix escenari.

In [ ]:
with open(TRAINING_INFO, 'r') as f:
    training_info = json.load(f)

SEGMENTATION = training_info['SEGMENTATION'] #utilitzam els mateixos hiperparàmetres que a l'entrenament
CROPPING = training_info['CROPPING']
SIZE = training_info['img_size']

In [ ]:
# !mkdir -p ~/.kaggle
# !mv /content/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
metadates = pd.read_csv('/content/sample_data/HAM10000_metadata.csv')
metadates = metadates.sort_values(by='image_id')

In [ ]:
class Formes(Dataset):

  def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform
        self.greyscale = False

  def __len__(self):
        return self.len

  def __setmasks__(self,masks):
      self.masks = masks

  def __getmask__(self,i):
      return self.masks[i]

  def __lenmasks__(self):
    return len(self.masks)

  def __addlabel__(self,label):
    self.labels = np.append(self.labels,label)

  def __addPath__(self,path):
    self.paths = np.append(self.paths,path)

  def __getdist__(self):
    return pd.Series(self.labels).value_counts()

  def __getlabels__(self):
    classes = [[],[],[],[],[],[],[]]
    for i in range(len(self.labels)):
      classes[self.labels[i]].append(i) #afegim l'index
    return classes

  def __setgreyscale__(self,mode):
    self.greyscale = mode

  def __getpath__(self,index):
    return self.paths[index]

  def __getitem__(self, index):
      path = self.paths[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_GRAYSCALE if self.greyscale else cv2.IMREAD_COLOR)

      if SEGMENTATION:
        mask = cv2.imread(self.masks[index], cv2.IMREAD_GRAYSCALE)
        _, mask_binaria = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
        image = cv2.bitwise_and(image, image, mask=mask_binaria)

        non_black_pixels = np.where(image > 0)

        x_min, x_max = np.min(non_black_pixels[1]), np.max(non_black_pixels[1])
        y_min, y_max = np.min(non_black_pixels[0]), np.max(non_black_pixels[0])

        marge = 10
        if ((x_min - marge) >= 0):
            x_min = x_min - marge
        if ((x_max + marge) <= SIZE):
            x_max = x_max + marge

        if CROPPING:
          image = image[y_min:y_max, x_min:x_max]

        image = cv2.resize(image, (SIZE, SIZE), interpolation=cv2.INTER_CUBIC)


      if self.greyscale:
        image = Image.fromarray(image, mode="L")
      else:
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      image = self.transform(image)

      return image, label

In [ ]:
def load_sets(transformation_training,transformation_default,training_dist,valitation_dist,testing_dist):

  illnes_dictionary = {
      'nv': 'Melanocytic nevi',
      'mel': 'Melanoma',
      'bkl': 'Benign keratosis-like lesions ',
      'bcc': 'Basal cell carcinoma',
      'akiec': 'Actinic keratoses and intraepithelial carcinoma / Bowens disease',
      'vasc': 'Vascular lesions',
      'df': 'Dermatofibroma'
  }

  img_files_1 = sorted(glob('/content/sample_data/HAM10000_images_part_1/*'))
  img_files_2 = sorted(glob('/content/sample_data/HAM10000_images_part_2/*'))
  img_files = img_files_1 + img_files_2

  img_files = np.array(img_files)

  mask_files = sorted(glob('/content/sample_data/HAM10000_segmentations_lesion_tschandl/*'))

  imgid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in img_files}

  mask_path_dict = {os.path.splitext(os.path.basename(x))[0].replace('_segmentation', ''): x for x in mask_files}

  metadates['path'] = metadates['image_id'].map(imgid_path_dict.get)
  metadates['mask_path'] = metadates['image_id'].map(mask_path_dict.get)

  metadates['illness'] = metadates['dx'].map(illnes_dictionary.get)
  metadates['illness_code'] = metadates['dx'].map({'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6})

  #Aquest illness_code s'utilitzarà com a label de la enfermetat

  img_number = len(img_files)

  X = metadates.drop('illness_code',axis= 1)
  y = metadates['illness_code']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testing_dist, random_state=42, stratify=y)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=valitation_dist/(training_dist+valitation_dist), random_state=42, stratify=y_train)

  train_data = Formes(X_train['path'].to_numpy(),y_train.to_numpy(),transformation_training)
  test_data = Formes(X_test['path'].to_numpy(),y_test.to_numpy(),transformation_default)
  validation_data = Formes(X_val['path'].to_numpy(),y_val.to_numpy(),transformation_default)

  if SEGMENTATION:

    train_data.__setmasks__(X_train['mask_path'].to_numpy())
    test_data.__setmasks__(X_test['mask_path'].to_numpy())
    validation_data.__setmasks__(X_val['mask_path'].to_numpy())

  return train_data, validation_data,test_data



In [ ]:
def genera_embeddings(model,dataset,activation,features_file):
  total = dataset.__len__()
  np.set_printoptions(threshold=np.inf)

  data = [["imatge","cnnoutput","fcoutput","path","label"]]

  f = WEIGHTS_PATH+features_file

  # Camps que guardarem, FC Input és el vector de característiques
  headers = ["Image Name", "FC Input", "Image Path", "Label"]

  with open(f, mode="w", newline="") as file:
      writer = csv.writer(file, delimiter=',')
      writer.writerow(headers)

  for i in range(total):
      imatge, label = dataset.__getitem__(i)
      imatge = imatge.to(device)
      imatge = imatge.unsqueeze(0)

      img_path = dataset.__getpath__(i)
      img_name = img_path.split('/')[-1]

      with torch.no_grad():
        output = model(imatge)

      #cnnoutput = activation['layer4'].flatten()
      fcoutput = activation['lastlayer'].flatten()

      data_row = [img_name, fcoutput.cpu().detach().tolist(), img_path, str(label)]

      with open(f, mode="a", newline="") as file:
          writer = csv.writer(file, delimiter=',')
          writer.writerow(data_row)  # Write row immediately

      # if i == 1:
      #   break

  print("Txt file writing completed.")

In [ ]:
mean = torch.tensor([194.57463374, 139.13953272, 145.36132088]) /255 #rgb
std = torch.tensor([35.92275236, 38.90347617, 43.33101831]) / 255

TRAINING = 0.80
VALIDATION = 0.10
TESTING = 0.10

transform = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std)
])

transform_training = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std),
])

train_data,validation_data,test_data = load_sets(transform_training,transform,TRAINING,VALIDATION,TESTING)


#Aquí posam el millor model triat
inceptionv3 = models.inception_v3()
num_features = inceptionv3.fc.in_features
inceptionv3.fc = nn.Linear(in_features=num_features, out_features=7)

#Li passam el fitxer de pesos
inceptionv3.load_state_dict(torch.load(pt_file[0], weights_only=True))

#definim la funció per obtenir els vectors de característiques
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = input[0].detach()
    return hook

model = inceptionv3
model.fc.register_forward_hook(get_activation('lastlayer'))

model.to(device)

os.makedirs(WEIGHTS_PATH+'Features',exist_ok=True)

model.eval()
genera_embeddings(model,train_data,activation,"Features/training_features.txt")
genera_embeddings(model,test_data,activation,"Features/testing_features.txt")


/usr/local/lib/python3.11/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Txt file writing completed.
Txt file writing completed.
